# Running an Inference Web Service on Colab
This notebook clones the entire repository on Google Colab and launches the inference service as a web application.

In [ ]:
%%capture
%%sh
git clone https://github.com/swaynos/urban-palm-tree.git
pip install aiofiles dotenv huggingface_hub ultralytics Flask

In [ ]:
%cd urban-palm-tree/src

In [ ]:
import asyncio
from flask import Flask, request, jsonify
from inference.yolo_object_detector import YoloObjectDetector
from inference.rush_inference import parse_rush_model_results
from utilities import config
from PIL import Image

# Create a Flask app
app = Flask(__name__)

PORT = 6969

# Initialize the detector
detector = YoloObjectDetector(config.HF_RUSH_DETECTION_PATH, config.HF_RUSH_DETECTION_FILENAME)

# Define a route for the inference
@app.route('/detect', methods=['POST'])
def detect_objects():
    # Check if an image file was provided
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        # Open the image file
        image = Image.open(file)

        # Run the detector. Keep in mind this is a syncrhonous operation
        results = asyncio.run(detector.detect_objects(image, parse_rush_model_results))

        # Return the results as JSON
        return jsonify(results), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    
# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=PORT)

In [ ]:
# If running on Colab
!pip install pyngrok
from google.colab import userdata
from pyngrok import ngrok
import os

NGROK = userdata.get('NGROK')  # Replace with your ngrok token
ngrok.set_auth_token(NGROK)

# Define a function to start Flask
def run_app():
    os.system(f"python3 -m flask run --host=0.0.0.0 --port={PORT}")

# Start Flask in the background
import threading
threading.Thread(target=run_app).start()

# Step 5: Set up ngrok tunnel
public_url = ngrok.connect(PORT)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{PORT}\"")

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=PORT)